
# Lumbar Spine Degeneration Classification with ResNet-50

This project applies deep learning to classify lumbar spine degeneration from MRI images using a fine-tuned ResNet-50 model.
The goal is to assist in automated diagnostic processes, which can help medical professionals by providing preliminary assessments.

## Features
- **Dataset**: RSNA Lumbar Spine Dataset.
- **Model**: Pre-trained ResNet-50, fine-tuned for multi-class classification.
- **Objective**: To classify spinal degeneration into distinct categories.

## Objectives:
1. **Data Understanding and Preparation**: Load and preprocess MRI images for model training.
2. **Model Development**: Use a pre-trained ResNet-50 as the backbone and fine-tune it for classification.
3. **Training and Optimization**: Train the model using advanced optimization techniques to handle data imbalance.
4. **Evaluation**: Measure performance using appropriate metrics and analyze results.



## Data Loading and Preprocessing

### Steps:
1. **Load Dataset**: Import MRI images and associated labels.
2. **Handle Class Imbalance**: Apply oversampling, undersampling, or data augmentation to balance classes.
3. **Data Augmentation**: Use transformations such as rotation, flipping, and scaling to enhance generalization.
4. **Prepare for Model Input**: Resize images to match the input size of ResNet-50 and normalize pixel values.

---


In [ ]:
import numpy as np
import pandas as pd
import os

From a different run, I found the files that do not have images in them but are called out in the dataset and saved the in a csv file

In [ ]:
missing_files_df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/missing_files.csv')
missing_files = missing_files_df['study_id'].to_numpy()

Load the datasets

In [ ]:
#Load the csv file to data frames
train_labels = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train.csv")
train_coordinates = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification//train_label_coordinates.csv")
series_descriptions = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification//train_series_descriptions.csv")

In [ ]:
print("Lenghth of train_labels:", train_labels.shape[0])
print("Lenghth of train_coordinates:", train_coordinates.shape[0])
print("Lenghth of series_descriptions:", series_descriptions.shape[0])

In [ ]:
missing_train_labels = train_labels[train_labels.isnull().any(axis=1)]['study_id']
missing_train_labels = missing_train_labels.reset_index(drop=True)

In [ ]:
#This cell is to find that data that is not shared between the datasets
study_id_train_labels = train_labels['study_id'].to_numpy()
study_id_train_coordinates = train_coordinates['study_id'].unique()

#Find the values in study_id_train_labels that are not in study_id_train_coordinates
not_in_coordinates = np.setdiff1d(study_id_train_labels, study_id_train_coordinates)

#Find the values in study_id_train_coordinates that are not in study_id_train_labels
not_in_labels = np.setdiff1d(study_id_train_coordinates, study_id_train_labels)

Drop the cases that have missing data from all data sets

In [ ]:
#drop the data that is not shared from the datasets
train_labels = train_labels.drop(train_labels[train_labels['study_id'].isin(not_in_coordinates)].index)
train_labels = train_labels.drop(train_labels[train_labels['study_id'].isin(not_in_labels)].index)
train_labels = train_labels.drop(train_labels[train_labels['study_id'].isin(missing_train_labels.to_numpy())].index)
train_labels = train_labels.drop(train_labels[train_labels['study_id'].isin(missing_files)].index)
train_labels = train_labels.reset_index(drop=True)

train_coordinates = train_coordinates.drop(train_coordinates[train_coordinates['study_id'].isin(not_in_coordinates)].index)
train_coordinates = train_coordinates.drop(train_coordinates[train_coordinates['study_id'].isin(not_in_labels)].index)
train_coordinates = train_coordinates.drop(train_coordinates[train_coordinates['study_id'].isin(missing_train_labels.to_numpy())].index)
train_coordinates = train_coordinates.drop(train_coordinates[train_coordinates['study_id'].isin(missing_files)].index)
train_coordinates = train_coordinates.reset_index(drop=True)

print("Lenghth of train_labels:", train_labels.shape[0])
print("Lenghth of train_coordinates:", train_coordinates.shape[0])


Here, I created a melted dataframe of the train_labels to be able to compare it with the train_coordinates and check for any additional missing data

In [ ]:
#Create labels for the images
valuevar = train_labels.columns
fvaluevar = valuevar.drop('study_id')

#Step 1: Melt the dataframe
melted_df = pd.melt(train_labels, id_vars=['study_id'],
                    value_vars=fvaluevar,
                    var_name='condition', value_name='severity')

#Step 2: Create the row_id by concatenating study_id and condition
melted_df['row_id'] = melted_df['study_id'].astype(str) + '_' + melted_df['condition']

#Step 3: Reorder columns, putting row_id at the front
final_df = melted_df[['row_id','study_id', 'condition','severity']]
final_df.head()

I manually checked for any addtional missing data and saved them is a csv file

In [ ]:
additional_drop = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/Addtional_drop.csv")
additional_drop_labels = additional_drop['Drop'].to_numpy()

Sort the data sets to allign all the data correctly

In [ ]:
final_df = final_df.drop(final_df[final_df['row_id'].isin(additional_drop_labels)].index)
final_df = final_df.sort_values(by=['study_id', 'condition'], ascending=[True, True])
final_df = final_df.reset_index(drop=True)

In [ ]:
final_df2 = final_df.drop(columns=['row_id','condition', 'study_id'])
img_labels = final_df2.to_numpy()

In [ ]:
np.save("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_labels.np", img_labels)

In [ ]:
train_coordinates = train_coordinates.sort_values(by=['study_id','condition','level'], ascending=[True, True, True])
train_coordinates = train_coordinates.reset_index(drop=True)

Check if the datasets have the same size

In [ ]:
print("Labels length:", len(final_df))
print("Coordinates length:", len(train_coordinates))

Extract the images from their files and save them in an array

In [ ]:
import os
import pydicom
import cv2

#Function to load the images
def load_images(image_path, img_size=(224,224)):
    dicom = pydicom.dcmread(image_path)
    img = dicom.pixel_array
    img = cv2.resize(img, img_size) #resize image for the model
    img = img/np.max(img) #normalize image pixels
    return img

In [ ]:
from tqdm import tqdm

#array for images
train_images = []

#Initialize a list to record study_ids with missing files
missing_files = []

#Loop over each study and use the serie_id and instance_number to load the image
for i, study_id in tqdm(enumerate(train_coordinates['study_id']), total=len(train_coordinates)):
    # Construct the file path
    path = "/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images/" + str(study_id) + "/" + str(train_coordinates.iloc[i][1]) + "/" + str(train_coordinates.iloc[i][2]) + ".dcm"
    train_images.append(load_images(path))

np.save("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_array.np", train_images)

In [ ]:
import numpy as np
import h5py

#train_images = np.load("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_array.np.npy")
labels = np.load("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_labels.np.npy")

In [ ]:
train_images = np.expand_dims(train_images, axis=-1)

In [ ]:
import numpy as np
import h5py

# Define the path to the saved array (assumes the images are already saved in Google Drive)
image_array_path = "/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_array.np.npy"

# Create a memory-mapped array with the target shape (35957, 224, 224, 3)
memmap_file = np.memmap('train_images_memmap.dat', dtype='float32', mode='w+', shape=(35957, 224, 224, 3))

# Load the original array (in chunks if possible) and copy it into the memmap array, while adding the third channel
original_images = np.load(image_array_path)  # Loading the (35957, 224, 224) array

for i in range(original_images.shape[0]):
    # Add the third channel (grayscale images) and store it in the memmap array
    memmap_file[i] = np.stack([original_images[i]] * 3, axis=-1)

# Flush the changes to disk
memmap_file.flush()

Use data data augmentation to increase the diversity of the training set.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Fit the generator to the training images
datagen.fit(train_images)

In [ ]:
#Apply elastic deformations
from imgaug import augmenters as iaa
seq = iaa.Sequential([iaa.ElasticTransformation(alpha=50, sigma=5)])
#Fit the deformations to the training images
train_images = seq(images=train_images)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.imshow(train_images[62])
plt.show()

Split the data for training and validation

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
# Load the memory-mapped array (without loading it entirely into RAM)
memmap_file = np.memmap("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_memmap.dat", dtype='float32', mode='r', shape=(35957, 224, 224, 3))

# Use this array during training
print(memmap_file.shape)

In [ ]:
train_images = np.array(memmap_file)

In [ ]:
labels = np.load("/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/train_images_labels.np.npy", allow_pickle=True)

label_mapping = {"Normal/Mild": 0, "Moderate": 1, "Severe": 2}

labels = np.array([label_mapping[label[0]] for label in labels])

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

train_images = preprocess_input(train_images)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
# Convert the labels to categorical format (if necessary)
y_train = to_categorical(y_train, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)

In [ ]:
import gc

del memmap_file
del train_images
del labels
gc.collect()

In [ ]:
#Create temp storage path
temp_storage_path = '/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/training_temp/'

#Create the directory if it doesn't exist
os.makedirs(temp_storage_path, exist_ok=True)

## Model Architecture

The model leverages **ResNet-50**, a widely used pre-trained convolutional neural network for feature extraction.
Key modifications include:
- Replacing the final fully connected layer to match the number of output classes.
- Adding dropout and L2 regularization to mitigate overfitting.

---


## Training and Optimization

### Steps:
1. **Loss Function**: Use focal loss to handle class imbalance effectively by prioritizing hard-to-classify samples.
2. **Optimizer**: Employ the Adam optimizer for adaptive learning rate adjustments.
3. **Early Stopping and Checkpoints**: Monitor validation loss to save the best-performing model.
4. **Augmented Training**: Train the model on the augmented dataset to improve robustness.

---


In [ ]:
import tensorflow as tf

#Define Focal Loss function to handle class imbalance
@tf.keras.utils.register_keras_serializable()
def focal_loss(gamma=2., alpha=[0.2, 0.5, 0.7]):
  @tf.keras.utils.register_keras_serializable()
  def focal_loss_fixed(y_true, y_pred):
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    y_true = tf.cast(y_true, tf.float32)
    cross_entropy = -y_true * tf.math.log(y_pred)
    loss = alpha * tf.math.pow(1 - y_pred, gamma) * cross_entropy
    return tf.reduce_mean(loss, axis=-1)
  return focal_loss_fixed

In [ ]:
from sklearn.utils import class_weight

#Compute class weights
y_train_classes = np.argmax(y_train, axis=1)
class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_classes), y=y_train_classes)

#Convert the NumPy array to a dictionary
class_weights = dict(enumerate(class_weights_array))


#For Focal Loss
total_examples = 28140 + 5660 + 2157
num_classes = 3

alpha = [
    total_examples / (num_classes * 28140),  # Class 0
    total_examples / (num_classes * 5660),   # Class 1
    total_examples / (num_classes * 2157)    # Class 2
]


In [ ]:
import tensorflow as tf

#Load the base ResNet50 model with pre-trained ImageNet weights, excluding the top layer
base_model = tf.keras.applications.ResNet50(
    weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#Freeze the layers of the base model
base_model.trainable = False

#Add custom layers for fine-tuning
x = base_model.output  # Output of ResNet50 model
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector per image
x = tf.keras.layers.Dense(1024, activation='relu')(x)  # Dense layer with ReLU activation , kernel_regularizer=tf.keras.regularizers.l2(0.01)
x = tf.keras.layers.BatchNormalization()(x)  # Add Batch Normalization
x = tf.keras.layers.Dropout(0.3)(x)  # Dropout for 3rd dense layer
output = tf.keras.layers.Dense(3, activation='softmax')(x)  # Final output layer with 3 classes

#Create the full model
model = tf.keras.Model(inputs=base_model.input, outputs=output)

#Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Data augmentation and loading using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64

#Prepare the data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
#val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

val_datagen.fit(X_val)

In [ ]:
#Save the model after every epoch in the temp storage file
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    temp_storage_path + 'model_checkpoint_adam.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

#Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

#Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
#Fine tune the model
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    steps_per_epoch= len(X_train) // batch_size,
    epochs=50,
    validation_steps= len(X_val) // batch_size,
    class_weight=class_weights,
    callbacks=[checkpoint, reduce_lr, early_stopping],
    verbose=1
)

In [ ]:
model.save('/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/freeze_resnet50_mri_model_Adam.keras')

## Evaluation and Metrics

### Metrics:
- **Accuracy**: Measure overall classification performance.
- **Precision, Recall, F1-Score**: Assess performance on individual classes, particularly imbalanced ones.
- **Confusion Matrix**: Visualize model performance across all classes.

---


In [ ]:
# Evaluate the model
score = model.evaluate(X_val, y_val)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
#Unfreeze the last 20 layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

#Recompile the model with lower lr
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Save the model after every epoch in the temp storage file
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    temp_storage_path + 'model_checkpoint_adam.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

#Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)

#Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
#Fine tune the model
history_tuned = model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    steps_per_epoch= len(X_train) // batch_size,
    epochs=50,
    validation_steps= len(X_val) // batch_size,
    class_weight=class_weights,
    callbacks=[checkpoint, reduce_lr, early_stopping],
    verbose=1
)

In [ ]:
# Save the fine-tuned model
model.save('/content/drive/MyDrive/ColabNotebooks/RSNA2024LumbarSpineDegenerativeClassification/unfreeze_resnet50_mri_model_Adam.keras')

In [ ]:
# Evaluate the model
score = model.evaluate(X_val, y_val)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## Results and Analysis  

**1. Training and Validation Trends**  
- The model achieved high test accuracy, indicating effective learning on the training data.  
- However, the validation accuracy was significantly lower, suggesting the model might not generalize well to unseen data.  

**2. Challenges**  
- **Class Bias**: Despite balancing the dataset, the model exhibited a strong bias toward the majority class (Class 0). This may stem from the inherent complexity of minority class patterns or insufficiently diverse augmentations.  
- **Overfitting**: The disparity between training/test accuracy and validation accuracy hints at potential overfitting to the training data.  

**3. Visualizations and Metrics**  
- **Loss Curves**: The training loss consistently decreased, but the validation loss showed fluctuations, further supporting the overfitting concern.  
- **Confusion Matrix**: The model predicted Class 0 with high confidence while misclassifying minority classes (Class 1 and Class 2) more frequently.  

**Key Observations**:  
- Precision and recall for the majority class were notably higher than for the minority classes.  
- Validation performance indicates the need for additional regularization techniques or improved data representation for minority classes.  

**Summary**:  
The model's bias toward the majority class highlights a need for further experimentation. Advanced techniques such as focal loss tuning, minority class oversampling with diverse augmentations, or even semi-supervised learning could address this issue. Additionally, incorporating cross-validation might provide better insights into generalization performance.  


## Conclusion and Future Work

This project successfully demonstrates the application of deep learning to classify lumbar spine degeneration.
Future improvements could include:
- Exploring additional architectures (e.g., EfficientNet or Vision Transformers).
- Applying semi-supervised learning to leverage unlabeled data.
- Conducting hyperparameter tuning to further optimize performance.

---



## Usage

1. Clone the repository:
   ```bash
   git clone https://github.com/AbduBarakat/lumbar-spine-classification.git
   cd lumbar-spine-classification
   ```

2. Install dependencies:
   ```bash
   pip install -r requirements.txt
   ```

3. Place your dataset in the `data/` folder.

4. Run the notebook:
   ```bash
   jupyter notebook lumbar_spine_classification.ipynb
   ```
